In [1]:
## Installation du FrameWork DEAP
!pip3 install deap
!pip3 install tensorflow
!pip install keras
#Import d'outils pythons
import random
import operator
import math
import pandas as pd
import numpy 


#Import Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense

##Import les outils deap
from deap import base
from deap import creator
from deap import tools
from deap import gp
from deap import algorithms
from math import sqrt

/bin/bash: pip : commande introuvable


In [2]:
#Import dataset
df =  pd.read_csv("heart_failure_clinical_records_dataset.csv")

In [3]:
# Preview the first 5 lines of the loaded data
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [4]:
# provide information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [5]:
from sklearn.preprocessing import StandardScaler
#On le converti en binaire
cols = ['age','creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
scaler = StandardScaler()
df[cols] = scaler.fit_transform(df[cols])

In [6]:
X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT'].copy()
from sklearn.model_selection import train_test_split
#ON split le dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test,y_test,test_size=0.5,stratify=y_test)

In [7]:
print('training samples:',len(X_train))
print('testing samples:',len(X_test))
print('validation samples:',len(X_val))

training samples: 239
testing samples: 30
validation samples: 30


In [8]:
X_train

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
151,0.098199,0,-0.569691,1,1.854958,1,-0.198258,-0.478205,0.539054,1,0,-0.171143
262,0.350833,1,-0.334316,1,-1.107370,0,-0.669374,0.005926,-1.731046,1,0,1.351746
130,-0.659702,1,1.265827,0,1.854958,1,-0.147050,-0.671857,0.312044,1,1,-0.313107
54,-0.070223,1,-0.332251,1,-0.007077,0,-0.085600,0.780535,-1.050016,0,1,-1.100363
166,-0.659702,0,-0.398321,0,1.854958,0,-0.444058,-0.671857,-0.823006,1,1,0.048257
...,...,...,...,...,...,...,...,...,...,...,...,...
228,0.350833,0,-0.542850,0,-1.107370,0,-0.269950,3.491668,-1.504036,0,0,0.990383
158,2.035057,1,0.338776,0,1.008578,0,-0.290433,-0.090900,-0.595996,1,0,-0.119519
295,-0.491279,0,1.278215,0,-0.007077,0,0.068025,-0.187726,0.539054,0,0,1.816357
252,-0.912335,0,-0.347736,0,0.585389,1,0.108991,-0.381379,-0.823006,1,0,1.093630


In [9]:
def individu():
    couches = []
    nbcouche = random.randrange(1,8)
    nbneurones = 12
    for k in range(nbcouche):
        w = []
        for j in range(nbneurones):
            a = [j,round(random.uniform(0,1),2)]
            w.append(a)
        couches.append([k,w])
    return couches

In [10]:
individu()

[[0,
  [[0, 0.7],
   [1, 0.4],
   [2, 0.33],
   [3, 0.89],
   [4, 0.98],
   [5, 0.0],
   [6, 0.05],
   [7, 0.04],
   [8, 0.29],
   [9, 0.34],
   [10, 0.44],
   [11, 0.73]]],
 [1,
  [[0, 0.9],
   [1, 0.64],
   [2, 0.97],
   [3, 0.03],
   [4, 0.45],
   [5, 0.45],
   [6, 0.6],
   [7, 0.57],
   [8, 0.78],
   [9, 0.3],
   [10, 0.7],
   [11, 0.91]]],
 [2,
  [[0, 0.92],
   [1, 0.43],
   [2, 0.48],
   [3, 0.62],
   [4, 0.51],
   [5, 0.85],
   [6, 0.42],
   [7, 0.75],
   [8, 0.99],
   [9, 0.89],
   [10, 0.25],
   [11, 0.99]]],
 [3,
  [[0, 0.45],
   [1, 0.79],
   [2, 0.29],
   [3, 0.12],
   [4, 0.36],
   [5, 0.73],
   [6, 0.47],
   [7, 0.32],
   [8, 0.07],
   [9, 0.7],
   [10, 0.85],
   [11, 0.92]]],
 [4,
  [[0, 0.8],
   [1, 0.04],
   [2, 0.2],
   [3, 0.68],
   [4, 0.69],
   [5, 0.66],
   [6, 0.4],
   [7, 0.43],
   [8, 0.25],
   [9, 0.09],
   [10, 0.49],
   [11, 0.79]]],
 [5,
  [[0, 0.17],
   [1, 0.1],
   [2, 0.82],
   [3, 0.63],
   [4, 0.11],
   [5, 0.61],
   [6, 0.34],
   [7, 0.63],
   [8, 0.5

In [11]:
def mutation(ind):
    couches = ind
    ##On va inserer une couche de neurone
    nbneurones = random.randrange(0,12)
    w = []
    for j in range(nbneurones):
        a = [j,round(random.uniform(0,1),2)]
        w.append(a)
    couches.append([len(couches),w])
    return couches,

In [12]:
def create_model(ind):
    model = Sequential()
    #Input
    model.add(Dense(12,input_dim=12,activation='sigmoid'))
    #Couche cachée
    for i in range(len(ind)):
        model.add(Dense(ind[0][1][i][1],activation='sigmoid'))
    ##Couche output
    model.add(Dense(1,activation='sigmoid'))
    return model

In [13]:
def fitness(ind):
    #print(X_test, y_test)
    #On génére notre model de couche en fonction de l'ind
    model = create_model(ind)
    model.compile(optimizer='sgd', loss='mean_squared_error', metrics=['mse'])
    # fit the keras model on the dataset
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=10)
    
    res = 1 / ( model.evaluate(X_test,y_test)[0] + 1 )
    return res,

In [14]:
#Crossover provisoire ...
##Mon crossover provoque un bug que je n'ai pas reussi a debug donc j'en ai fait un de provisoir
def crossover(ind1,ind2):
    return ind1, ind1

In [15]:
"""
def crossover(ind1,ind2):
    dim1 = len(ind1)
    dim2 = len(ind2)
    newind = []
    if dim1 > dim2 : 
        for i in range(dim2):
            couche = []
            if ind1[i][0] == ind2[i][0]:
                couche.append(ind1[i][0])
                for j in range(len(ind1)):
                    couche.append(ind1[i][1][j])
                for k in range(len(ind2)):
                    couche.append([len(ind1)+k,ind2[i][1][k][1]]) 
                newind.append(couche)
        return newind, newind
    else :
        for i in range(dim1):
            couche = []
            if ind1[i][0] == ind2[i][0]:
                couche.append(ind1[i][0])
                for j in range(len(ind1)):
                    couche.append(ind1[i][1][j])
                for k in range(len(ind2)):
                    couche.append([len(ind1)+k,ind2[i][1][k][1]]) 
                newind.append(couche)
        return newind, newind
"""

'\ndef crossover(ind1,ind2):\n    dim1 = len(ind1)\n    dim2 = len(ind2)\n    newind = []\n    if dim1 > dim2 : \n        for i in range(dim2):\n            couche = []\n            if ind1[i][0] == ind2[i][0]:\n                couche.append(ind1[i][0])\n                for j in range(len(ind1)):\n                    couche.append(ind1[i][1][j])\n                for k in range(len(ind2)):\n                    couche.append([len(ind1)+k,ind2[i][1][k][1]]) \n                newind.append(couche)\n        return newind, newind\n    else :\n        for i in range(dim1):\n            couche = []\n            if ind1[i][0] == ind2[i][0]:\n                couche.append(ind1[i][0])\n                for j in range(len(ind1)):\n                    couche.append(ind1[i][1][j])\n                for k in range(len(ind2)):\n                    couche.append([len(ind1)+k,ind2[i][1][k][1]]) \n                newind.append(couche)\n        return newind, newind\n'

In [16]:
#Création des individus
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, individu)

#Création de la population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", crossover)
toolbox.register("mutate", mutation)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", fitness)

In [17]:
pop = toolbox.population(n=10)

In [18]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
mstats = tools.MultiStatistics(fitness=stats_fit)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

NGEN = 10
CXPB = 0.4
MUTPB = 0.2

hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, CXPB, MUTPB, NGEN, stats=mstats,
                               halloffame=hof, verbose=True)

Epoch 1/5
24/24 [==============================] - 0s 4ms/step - loss: 0.2500 - mse: 0.2500 - val_loss: 0.2500 - val_mse: 0.2500
Epoch 2/5
24/24 [==============================] - 0s 801us/step - loss: 0.2500 - mse: 0.2500 - val_loss: 0.2500 - val_mse: 0.2500
Epoch 3/5
24/24 [==============================] - 0s 822us/step - loss: 0.2500 - mse: 0.2500 - val_loss: 0.2500 - val_mse: 0.2500
Epoch 4/5
24/24 [==============================] - 0s 794us/step - loss: 0.2500 - mse: 0.2500 - val_loss: 0.2500 - val_mse: 0.2500
Epoch 5/5
1/1 [==============================] - 0s 752us/step - loss: 0.2500 - mse: 0.2500
Epoch 1/5
24/24 [==============================] - 0s 4ms/step - loss: 0.2500 - mse: 0.2500 - val_loss: 0.2500 - val_mse: 0.2500
Epoch 2/5
24/24 [==============================] - 0s 815us/step - loss: 0.2500 - mse: 0.2500 - val_loss: 0.2500 - val_mse: 0.2500
Epoch 3/5
24/24 [==============================] - 0s 825us/step - loss: 0.2500 - mse: 0.2500 - val_loss: 0.2500 - val_mse: 0.

In [19]:
hof[0].fitness

##Ma fitness n'évolue pas et je pense que c'est parce que mon crossover n'est pas appliqué..

deap.creator.FitnessMax((0.8,))